### **quantum eigensolver approach**

In [1]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit.primitives import Estimator
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

### **Molecule**

In [2]:
%%time
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    #atom="O 0 0 0; H 0.757 0.586 0; H -0.757 0.586 0",
    #atom="C 0 0 0; H 0.629 0.629 0.629; H -0.629 -0.629 0.629; H -0.629 0.629 -0.629; H 0.629 -0.629 -0.629",
    #atom="C 0 0 0; C 1.54 0 0; H 0.629 0.629 0.629; H -0.629 -0.629 0.629; H -0.629 0.629 -0.629; H 2.169 0.629 0.629; H 0.911 -0.629 0.629; H 0.911 0.629 -0.629",
    #atom="C 0 0 0; C 1.54 0 0; C 3.08 0 0; H 0.629 0.629 0.629; H -0.629 -0.629 0.629; H -0.629 0.629 -0.629; H 2.169 0.629 0.629; H 0.911 -0.629 0.629; H 0.911 0.629 -0.629; H 3.709 0.629 0.629; H 2.451 -0.629 0.629; H 2.451 0.629 -0.629",
    #atom="C 0 0 0; C 1.54 0 0; C 3.08 0 0; C 4.62 0 0; H 0.629 0.629 0.629; H -0.629 -0.629 0.629; H -0.629 0.629 -0.629; H 2.169 0.629 0.629; H 0.911 -0.629 0.629; H 0.911 0.629 -0.629; H 3.709 0.629 0.629; H 2.451 -0.629 0.629; H 2.451 0.629 -0.629; H 5.249 0.629 0.629; H 3.541 -0.629 0.629; H 3.541 0.629 -0.629",
    #atom="C 0 0 0; C 1.54 0 0; C 3.08 0 0; C 4.62 0 0; C 6.16 0 0; H 0.629 0.629 0.629; H -0.629 -0.629 0.629; H -0.629 0.629 -0.629; H 2.169 0.629 0.629; H 0.911 -0.629 0.629; H 0.911 0.629 -0.629; H 3.709 0.629 0.629; H 2.451 -0.629 0.629; H 2.451 0.629 -0.629; H 5.249 0.629 0.629; H 3.541 -0.629 0.629; H 3.541 0.629 -0.629; H 6.789 0.629 0.629; H 5.081 -0.629 0.629; H 5.081 0.629 -0.629",
    #atom="C 0 0 0; C 1.54 0 0; C 3.08 0 0; C 4.62 0 0; C 6.16 0 0; C 7.7 0 0; H 0.629 0.629 0.629; H -0.629 -0.629 0.629; H -0.629 0.629 -0.629; H 2.169 0.629 0.629; H 0.911 -0.629 0.629; H 0.911 0.629 -0.629; H 3.709 0.629 0.629; H 2.451 -0.629 0.629; H 2.451 0.629 -0.629; H 5.249 0.629 0.629; H 3.541 -0.629 0.629; H 3.541 0.629 -0.629; H 6.789 0.629 0.629; H 5.081 -0.629 0.629; H 5.081 0.629 -0.629; H 8.329 0.629 0.629; H 6.621 -0.629 0.629; H 6.621 0.629 -0.629",

    basis="6-31g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

CPU times: user 19 ms, sys: 8.62 ms, total: 27.6 ms
Wall time: 42.6 ms


### **Electronic Structure problem**

In [3]:
%%time
problem = driver.run()
print(problem)

CPU times: user 28.8 ms, sys: 19.4 ms, total: 48.2 ms
Wall time: 96 ms


### **Initial State for Eigensolver**

In [4]:
%%time
initial_state = HartreeFock(
    problem.num_spatial_orbitals,
    problem.num_particles,
    JordanWignerMapper(),
)

CPU times: user 2.59 ms, sys: 1.04 ms, total: 3.63 ms
Wall time: 6.24 ms


### **Ansatz for Eigensolver**

In [5]:
%%time
ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    JordanWignerMapper(),
    initial_state=initial_state,
)

CPU times: user 16 ms, sys: 3.12 ms, total: 19.1 ms
Wall time: 18.1 ms


In [6]:
%%time
vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

CPU times: user 6.54 ms, sys: 1.31 ms, total: 7.85 ms
Wall time: 9.39 ms


<timed exec>:1: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.


In [7]:
%%time
calc = GroundStateEigensolver(JordanWignerMapper(), vqe_solver)

CPU times: user 7 μs, sys: 1 μs, total: 8 μs
Wall time: 9.3 μs


### **Eigensolver Result**

In [8]:
%%time
result = calc.solve(problem)
print(result)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.871583225266
  - computed part:      -1.871583225266
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.151614230817
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388949773787]
    - computed part:      [0.0  0.0  1.388949773787]
  > Dipole moment (a.u.): [0.0  0.0  -0.000001073787]  Total: 0.000001073787
                 (debye): [0.0  0.0  -0.000002729294]  Total: 0.000002729294
 
CPU times: user 2.64 s, sys: 40.9 ms, total: 2.68 s
Wall time: 2.69 s
